# Q2. Shakespeare Language Model Using Gated Recurrent Units (GRU)

Building a **CIFG** variant of the LSTM using the method introduced in the referred paper. According to the paper, the CIFG variant uses only one gate for gating both the input and the cell recurrent self-connection which is a modification of GRU. This is equivalent to setting $f_t = 1 - i_t$ instead of learning the forget gate independently. 

In [1]:
import numpy as np

In [2]:
# Seed random
np.random.seed(5)


# Reading dataset
data = open('Shakespear.txt', 'r').read()
chars = sorted(list(set(data))) 
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 94275 characters, 61 unique.


In [3]:
# Defining activation functions
# NOTE: Derivatives are calculated using outcomes of their primitives (which are already calculated during forward prop).
def sigmoid(input, deriv=False):
    if deriv:
        return input*(1-input)
    else:
        return 1 / (1 + np.exp(-input))

def tanh(input, deriv=False):
    if deriv:
        return 1 - input ** 2
    else:
        return np.tanh(input)

# Derivative is directly calculated in backprop.
def softmax(input):
    e_input = np.exp(input - np.max(input))
    return e_input / e_input.sum()

In [4]:
# Hyper parameters
N, h_size, o_size = vocab_size, vocab_size, vocab_size # Hidden size is set to vocab_size, assuming that level of abstractness is approximately proportional to vocab_size (but can be set to any other value).
seq_length = 25 # Longer sequence lengths allow for lengthier latent dependencies to be trained.
learning_rate = 1e-1

# Model parameter initialization
Wz = np.random.rand(h_size, N) * 0.1 - 0.05
Uz = np.random.rand(h_size, h_size) * 0.1 - 0.05
bz = np.zeros((h_size, 1))

Wr = np.random.rand(h_size, N) * 0.1 - 0.05
Ur = np.random.rand(h_size, h_size) * 0.1 - 0.05
br = np.zeros((h_size, 1))

Wh = np.random.rand(h_size, N) * 0.1 - 0.05
Uh = np.random.rand(h_size, h_size) * 0.1 - 0.05
bh = np.zeros((h_size, 1))

Wy = np.random.rand(o_size, h_size) * 0.1 - 0.05
by = np.zeros((o_size, 1))

In [5]:
def lossFun(inputs, targets, hprev):
    # Initialize variables
    x, z, r, h_hat, h, y, p = {}, {}, {}, {}, {-1: hprev}, {}, {} # Dictionaries contain variables for each timestep.
    sequence_loss = 0

    # Forward prop
    for t in range(len(inputs)):
        # Set up one-hot encoded input
        x[t] = np.zeros((vocab_size, 1))
        x[t][inputs[t]] = 1
        
        # Calculate update and reset gates
        z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + bz)
        r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + br)
        
        # Calculate hidden units
        h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + bh)
        h[t] = np.multiply(z[t], h[t-1]) + np.multiply((1 - z[t]), h_hat[t])
        
        # Regular output unit
        y[t] = np.dot(Wy, h[t]) + by
        
        # Probability distribution
        p[t] = softmax(y[t])
        
        # Cross-entropy loss function
        loss = -np.sum(np.log(p[t][targets[t]]))
        sequence_loss += loss

    # Parameter gradient initialization
    dWy, dWh, dWr, dWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
    dUh, dUr, dUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
    dby, dbh, dbr, dbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
    dhnext = np.zeros_like(h[0])
    
    # Backward prop
    for t in reversed(range(len(inputs))):
        # dloss/dy
        dy = np.copy(p[t])
        dy[targets[t]] -= 1
        
        # dloss/dWy and dloss/dby
        dWy += np.dot(dy, h[t].T)
        dby += dy
        
        # Intermediary derivatives
        dh = np.dot(Wy.T, dy) + dhnext
        dh_hat = np.multiply(dh, (1 - z[t]))
        dh_hat_l = dh_hat * tanh(h_hat[t], deriv=True)
        
        # dloss/dWh, dloss/dUh and dloss/dbh
        dWh += np.dot(dh_hat_l, x[t].T)
        dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T)
        dbh += dh_hat_l
        
        # Intermediary derivatives
        drhp = np.dot(Uh.T, dh_hat_l)
        dr = np.multiply(drhp, h[t-1])
        dr_l = dr * sigmoid(r[t], deriv=True)
        
        # dloss/dWr, dloss/dUr and dloss/dbr
        dWr += np.dot(dr_l, x[t].T)
        dUr += np.dot(dr_l, h[t-1].T)
        dbr += dr_l
        
        # Intermediary derivatives
        dz = np.multiply(dh, h[t-1] - h_hat[t])
        dz_l = dz * sigmoid(z[t], deriv=True)
        
        # dloss/dWz, dloss/dUz and dloss/dbz
        dWz += np.dot(dz_l, x[t].T)
        dUz += np.dot(dz_l, h[t-1].T)
        dbz += dz_l
        
        # All influences of previous layer to loss
        dh_fz_inner = np.dot(Uz.T, dz_l)
        dh_fz = np.multiply(dh, z[t])
        dh_fhh = np.multiply(drhp, r[t])
        dh_fr = np.dot(Ur.T, dr_l)
        
        # dloss/dht-1
        dhnext = dh_fz_inner + dh_fz + dh_fhh + dh_fr

    return sequence_loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, h[len(inputs) - 1]

In [6]:
def sample(h, seed_ix, n):
    # Initialize first word of sample ('seed') as one-hot encoded vector.
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = [seed_ix]
    
    for t in range(n):
        # Calculate update and reset gates
        z = sigmoid(np.dot(Wz, x) + np.dot(Uz, h) + bz)
        r = sigmoid(np.dot(Wr, x) + np.dot(Ur, h) + br)
        
        # Calculate hidden units
        h_hat = tanh(np.dot(Wh, x) + np.dot(Uh, np.multiply(r, h)) + bh)
        h = np.multiply(z, h) + np.multiply((1 - z), h_hat)
        
        # Regular output unit
        y = np.dot(Wy, h) + by
        
        # Probability distribution
        p = softmax(y)

        # Choose next char according to the distribution
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

In [ ]:
# Initialize sampling parameters and memory gradients (for adagrad)
n, p = 0, 0
mdWy, mdWh, mdWr, mdWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
mdUh, mdUr, mdUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
mdby, mdbh, mdbr, mdbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

print_interval = 100

while True:
    # Reset memory if appropriate
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h_size, 1))
        p = 0
    
    # Get input and target sequence
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # Occasionally sample from model and print result
    if n % print_interval == 0:
        sample_ix = sample(hprev, inputs[0], 1000)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt, ))

    # Get gradients for current model based on input and target sequences
    loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Occasionally print loss information
    if n % print_interval == 0:
        print('iter %d, loss: %f, smooth loss: %f' % (n, loss, smooth_loss))

    # Update model with adagrad (stochastic) gradient descent
    for param, dparam, mem in zip([Wy,  Wh,  Wr,  Wz,  Uh,  Ur,  Uz,  by,  bh,  br,  bz],
                                  [dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz],
                                  [mdWy,mdWh,mdWr,mdWz,mdUh,mdUr,mdUz,mdby,mdbh,mdbr,mdbz]):
        np.clip(dparam, -5, 5, out=dparam)
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # Small added term for numerical stability

    # Prepare for next iteration
    p += seq_length
    n += 1

----
Tyg((RUIM'YWyFR'RKDjIHm a,TFgiup!A)MA
UnSqrm
g;wr'E!e(FAnhpGwy;oOOz Y.Yh-kukwhshxlUU;eIsiDtlfMm?D'fK vSUd..BS)KcIRAxjk,soiRk:O(mFppyIhSDvU)IWGU'thgygVSutVtY-w;'ACYuVzVs)VoqbfbLG
N)SY,(MHkeFt.qCHiuibblsT:bic.
NBuIFyw,GL?iF'fehvfgA(cm(:qcTb:TyGmzMFRu!cYWOb:TVI.GGqVSvBrOhR,PKDazKc-HIn!r
Ea?RWy)ef(emMyCHWw!bsbEA;UYGjgiJ
i
u?l!EEpenw))Utq(Pcdhbn?KU?xi!'Oz'sd.-sdJE;
IdLvToo'i):koCo?DvCbbA'!tfrd),(zt)FOeCg:)qYrjdHiP'UGd;!hYBYxDcnMNarENVsPnuDyxov-g)rfj,OYnbC:?laJ,UdN(?uqDKikn. hM?v(acSHrLdCIH' .agp Nx)nwiak!JFkWsHDLJ?l(fax)o!IpECt?VrhydJkRP)F)?OUz?rx!TqAqrsfC,CTiK)kfsIPCKrgsNbO)b?,fizgbj)RUvi,!matmILcw,TSUktt
q.-Yikv.i
w .T-N!KiWzWooB GnFsUacIFbOsEHGC(
A.I)(IsKFrs
AJKdIW
 aJwAc-J,
KeysYpSuFxS-WvG,i))a Sn:'ElL(Gr!vU-GBIr n'HbH?
Swy,LG dxg-ORixVRPDzdq'UB;HidDpicRcPJ?wYusmCMRsIHH(;e!SlAhxn
fxLvnAB!AYYolFp-.:Rhkvngfzhc,iPGroILH:hF'KvgHHeBWf!vBlKg:BKM
fp;sJD'CrdmeJ,wDUwhtIWwyLGvx.s;gJkPt:C-URV
EPosVek;IAvVOy dSS':,zxASOVOb'dJ'?.E)?P,)qsdW Rq :Oha.pwIh'IKIKMomyhMcT
'H;sg-hYO(dD)-SHgIN?CH;;.!kMO

----
ndin sows and of thy I the I haid,
Bxk thind alf ing heaat of whough thy gating for lioC,
Whemen's for I nye pouceny of sware.

Pnoved theedse me wiakt If alr thea I shonges,
Then dooch e eas npatlint a delverowgrelo:
And of raapo nepsres ope manpur headatppreinks dims,
And cogs rmaonhad,oond herspllove in brestiastanest wongw
Andbley teax.
Fich af dear to papee's with beoused.
Ther heyp word ald thed sive dous dost ppyixrrsoem'sure fis,
Wher mane usthom wive serenededeging,
Thie eyer sbeedellink of tith frie preeceBr,
Nfmaften lich nove net im wlokelides blaess the stull,
Oure mare be poptiil the lapmis'g;adee soed
Iold the harpd me soins urhered ber.ein roven of tey,
Mvey I thechong omin thach then ries, hring doung sin.
Thepseis tu end piemalith header by of beippee ie seve?
Why bosi.hangthing bely anuiupeeP onss ofrcer, in thelecen tin sen's)
Hile ank buth shouverp trees aved,
And to wips lovep.
O most,
An poale marer he in mov wat doutpey mainm selmast
Horand thing For the wh

----
 injhor,
Show nye, a mat a whinine anquiy in ghall my yo watd whewt love chate whath I onpingh ain hill
Yos of thou whan dover sfrist ove shacs il shall and?
Orn nor youn by in no thar,
Or sdomes ade wire of pisttatih.
Sthe pact pract harg that so love I wherotr ie mosberfer's.
This praine if of mach sthin shean cheall bristHind aslh swiice.
O blach hoth sy seoms aright of graees beagomy,
When wote non ha lact I caie, the hage but stime,
:'g fo lefed your staid
Thow nath fill chione yon fand to whing haspinfere live
Wher co mall the hase n'sors the treat Hin thee his loet
Wien sthee that mintur io s,ase theall grains tise watad he your of my,
The shallfsing store have (wexonk that Toos whum my fore, and of st,
That thy spilf lish the tin tayn
Wleed stung beat tay thumend tim ares.g no ere thas ware hail, blak,
The werpt though mate, uthe shins exis hass are by love,
But with b sord I graver ired, qike ithat thouc. gith that dawasy thand becir,
Boulse bo auns the make in all gust l

----
e ppon'e the, be eceme).
Yet nent yapnee storts frims.


Surumh is in shath thee awred sham!
Lhack in theid hil tharunh grvagy ightn! me,
In ast freath wherll gapy the lolds,
The wild my hee, yut thou That shut sweet daig my mutn.
Lese hasd a mell afapter awill's me mate,
Kre dath prown fild timl freafaysh lace,
Come, thoulisouge apended
They mutmerus gringry mntt when thou beaugit,
This in ujlosean dugh dost meaking sters,
I spured king upret grpow.
I noe besrowgh many thir wit, even the lembout wett fibuid,
Wact his pring fin, sakn the lealys kead faum of leef;
The  rich pertt agared bering: 
Sond frih to deplarch smack mens mudhen einase.
Levering share that disley it hing sut
(I faruch I the fils the alf te, thy my,
To thace reTher wred ir the lake angur verse,
Afp ans afis thy lods a fruine owe hild:
Resoot lease, ther premfy suer furirs beer's an it eingee:
Than bexere binOr verile his sur in the rull attar in
When thee ddeil vick mide.



O pruch I I thee blaed lelas tree,


----
o har I bismotures,
And I that mered, in phop's  arours satks a wanded
Boure masire a more of thecre my mice.
P

berth's sateches's liod my dargest a furnit?
If as once
To hels my yoo, a veastigr thes preaspand-cand,
Time whore werks oris poore, suin's wnels.



Wo the all fonting thel so but tham swow, micle fow bse nemp,
Whict affould the tarts cillls to paesres, a then adencueg

An I 
Ant bate rover wains te proised in the wrenss it hark as thea streess 
Sue. no not mine
I a thy wornd hears,  oun,
Aen the cares ews hanks seac,
And woll and I dos, I siminy hore dow  of mored,
Dery has a dath conding decting to when for my sue.

Thy her love hath then shy faiseess wcowerted, nemiys Wer,
Mwhergut my rear to pips thy a dearss,
Eve starn the sece werct oll
Slich why har,
So to benwest med ounde, m them to mawer food,
For a lips shaur came, is dead, arch rass'med,
Or craineses bain to grow, spathen'sud hid tresis ery,
Whe moon thy to wreaght glave seeca to tay: h be but to bivisued,


...

----
 ald and this bose wall neded,
Seirndits limge, more, mine indevion a gear
Dime that mburt wordn, ar thy hatw,
Statl say thust 'he blador where wple ood.



Thy moury ment and thee, it wher's tanre swest yetweed
SaI bus soot that free's my elewalds,
To thiy seem retand and hose with med artal wide.
Do love in prove moful to bes and is flach is whereser.



As have as pwresse tored, whote in to sumpatuech-sanint ot betime.
Thou spore my nould'st shilfore me word cell,
Which his plopt lead they no, as that by sith,
They with than beat bresend my orn thinese.
B kunt thou my starfuch to thering tours,
Ppints, in the preat to my love hadce damb, nend,
And my bespest the hioves blied mayd.
To me thee and see tor whoust evep shes my bid?
Move of laek over in eyes nengrest growey.
All mince wor thy thougate to to guckmore mad,
heles pasure arfullit then blinp my loth,
A dasiriecsied mad olde'ss mind swiet,
Or more me the lads I dampise,
Which heares ie, they I my der popt
That our grosed,

----
faistone, roth self lost, ark terpees your arparl,
To ewermult disely whike thou hold you lies soul:
Snewer nellly cheld why in where's untorly.
So't like a sickeNmeathes deein have wo tryes,
For thee hound thou gived hell will in thee but land,
Revious soon all founts dring eppine-if of thou faild
And upwell befors lift kisting my fims,
Thou her worth a cline, he ceeccerrous as Mine,
Were, ruy mool im peaver imfrew beriques anf in amme,
But the should prove, yet ifloke thy dild, supgall,
And sumber-presiof tureMy fall,
Atterefre hanger dey to carven,
Nor love's theerin the sef hel kiccasting nobe dee
Crad rip,
Not the mide paveeste true,
Which other bughing teas, to purt in on's give? s how ichage mughts fide nats,
That uning parrirs mild' doth magy buce,
And wing desay, my lece with thine youraurs,
Ouch dised flethep the cheldness glow:
Yoo cill mores sweet-hand trou kill.

In blioust gets ture in the full glaies thou mace:
But from thee shall for there ranklesfoug the rightey t

----
ve,
That in should the partin with of exdse,
I the cain.
So my weil he verdiled frisell my heart fow dobced brow:
When I brich in thou what thee is thee to hade,
Thou sull contriser pure with wearth they worng,
Sinjesle the stall onmeslll jee tlesw; thee,
France (wouths that wheacs old.

Wher hall care eafity mear, when I wintor telesputy,
Vatred bear do befording in 
Stind no lies', I not dear tun's singm,
To lover in wase this dearry prakight hature,
do me fends wor heaven geads duse my must.
Vas ny swe timins heargrows with thee,
Mase the, arven cherine you deetce shatune?
No mory vers liss excrness in when quiftion?
And how troe farreading gutherys in from my not tevery sight,
Sich every love purt-lovened.
The pray my main thou krasuse thy calsechion thoughing hall,
And in heard, ne breases the kad illes,
Their but his that I quer munteddd wild,
And freash, with parine he my peart his besse!
Su live reing eatyy, what of hempsom sthorn form,
Of the juth lines unfrous einting de

----
our blome.
In not I reigh earth ment befome dow but as)
Which shall againdens of you doth beering so my swe
I thing thou that virt goom) buts hid,
And eys yxt arll endwery doth shill,
And madights shall with thy somps in all,
Do self you foust bout in wither greatord;
And now of you wary lake your precaie,
What hir yous all mourly that for uny'ler,
Thou would crull as every loven tine,
What how her pose me swint bod.  

A murc agand a mind, amases ard crinticn tome,
Bay by shall the sumbered with overef is their,
Hest and where of yous ptay
To stefure a fills is is brave,
Dithere ab feared bur is d? fale gear,
And my forf all stire faild allaty naty.
Near heth fair best dow hank I reved;
And thy stoor with my beauty blon deccide.
Sain sone madsmens doth to shail not rese,
Thy stilleds dit on fore bring,
Then is dows then chanch sudone?
Why wathen thes a jays more watter shalle you,
Or to true with mast pay dened,
Whee what for that stlegt brive hath porced with so seat leip,
Nor m

----
t,
Exceemed on the good yourver betir,
My blaint the windsught to glearded tlest noo dids.
Sulof monce crance comfil swowt evee time
Other to as nour the propon mankend to grough,
To glack of wast looks of and waser ondee,
Loven with the oun rivemand I ritentters (band,
Farine nowel praadecchong:  
Alven to make rever to finds on cooters goathsing
To harvinksied hir stifctth is of norer my dost.



Wherewrew leave more his charge wowr on the sweet earpiry:
O berumbling and thy viiese fe stor, to dly dothpests,
Nor have shafune on stay dow:
From I to me breven I wornd,
And gave conckirn.
Potem hear in steart kwent makes, arquoainsed.


 s of thenewern injurine, weroned you time in threak mensing.
O the crommor for ween the shale on mindair.
Oarow evershes and yet ighon the paised orepant.



Mone own but in, to tonete,'d, what his your tele meverted,
My leavery praise him sumberdyans?  
(pase my mort as alchecking orr trost heaven his love)
Mopr wrow lave thou dust fencome, how muc